# Linear Algebra for Data Science. Final Test
## Prepared by Said Magomedov, group MFE221

In [1]:
%%html
<style>
  .rendered_html {
    font-family: "Palatino", serif;
  }
</style>

In [2]:
import numpy as np, matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy.linalg import expm
from sympy import Matrix
from numpy.testing import assert_almost_equal
from IPython.display import display, Math

## Problem 1
In order to find the best approximation matrix of rank r for given matrix I need to construct the SVD and take rxr leading principal submatrices instead. <br>
Finally the L2 norm of a bias matrix by definition must be equal to its largest singular value, i.e. $\sigma_{r+1}$, where $\sigma_i$ is a singular value of the Gram matrix of the given one. 

In [3]:
A = np.array([[-18,-22,-80,-32],[63,4,2,-34],[-54,-20,56,-4]])

In [4]:
# recall my function from the MidtermTest and modify it for the case of rank 2 
def svd2(A):
    eigvals, eigvecs = LA.eig(A.T@A)
    eigvals = np.where(eigvals < 0, 0, eigvals)
    sort_i = np.argsort(eigvals)[::-1]
    S = np.diag(np.sqrt(eigvals[sort_i]))
    V = eigvecs[:, sort_i]
    u0 = A@V[:, 0] / S[0,0]
    u1 = A@V[:, 1] / S[1,1]
    u2 = A@V[:, 2] / S[2,2]
    u3 = np.zeros(3)
    U = np.array([u0, u1, u2, u3]).T
    
    U2 = U[:,:2]
    S2 = S[:2,:2]
    V2 = V[:,:2]
    
    SVD2 = U2@S2@V2.T

    return [SVD2, U2, S2, V2.T]

In [5]:
matrices = svd2(A)
output = []
for matrix in matrices:
    matrix_str = "\\begin{pmatrix}\n"
    matrix_str += "\\\\\n".join([" & ".join(map(str, row)) for row in np.round(matrix, 3)])
    matrix_str += "\n\\end{pmatrix}"
    output.append(matrix_str)
    
display(Math(fr'A_1 = {output[1]} * {output[2]} * {output[3]} = {output[0]}'))

<IPython.core.display.Math object>

In [6]:
A1 = svd2(A)[0]
bias_norm = LA.norm(A-A1, ord=2)
sigma3 = np.sqrt(np.sort(LA.eigvals(A.T@A))[::-1][2])

assert_almost_equal(bias_norm, sigma3, decimal=3)

No assertion error raised, so everything was done correctly and $||A - A_1||_2 = 42$.

---
---

## Problem 2
Consider system $Ax = b$ and its aproximate analogue $\hat{A}\hat{x} = \hat{b}$ such that:
$$\Delta A = \hat{A} - A \approx 0, \quad \Delta b = \hat{b} - b \approx 0, \quad \Delta x = \hat{x} - x \approx 0$$

Relative errors are:
$$\delta A = \frac{||\Delta A||}{|A|}, \quad \delta b = \frac{||\Delta b||}{|b|}, \quad \delta x = \frac{||\Delta x||}{|x|}$$

General formula for the upper bound of the solution relative error:
$$\delta x \le \frac{\kappa(A)}{1-\kappa(A)\delta A}(\delta A + \delta b),$$ <br>

<center> where $\kappa(A) = ||A|| \cdot ||A^{-1}||$ is a conditional number of a matrix A <center>

In [7]:
A = np.array([[-3.92,-0.13],[5.03,-5.84]])
b = np.array([-3.99,-0.98])
x = LA.inv(A)@b

A_hat = np.round(A)
b_hat = np.round(b)
x_hat = np.round(LA.inv(A_hat)@b_hat) 
# need to round x_hat here since common Python floating number error raises and returns non-integer values

delta_A = A_hat - A
delta_b = b_hat - b
delta_x = x_hat - x

In [8]:
print(f'Approximate solution: ({x_hat[0]}, {x_hat[1]})')

Approximate solution: (1.0, 1.0)


For the L1 norm.

In [9]:
kappa_A1 = LA.norm(A, 1) * LA.norm(LA.inv(A), 1) 
# kappa_A1 = np.linalg.cond(A, 1)

re_A1 = LA.norm(delta_A, 1) / LA.norm(A, 1)
re_b1 = LA.norm(delta_b, 1) / LA.norm(b, 1)

re_x1_ub = (re_A1 + re_b1) * kappa_A1 / (1 - kappa_A1 * re_A1)

print(f'The upper bound of the relative error of x in the L1 norm is {re_x1_ub.round(3)}.')

The upper bound of the relative error of x in the L1 norm is 0.183.


For the L2 norm

In [10]:
kappa_A2 = LA.norm(A,ord=2) * LA.norm(LA.inv(A), 2)
# kappa_A2 = np.linalg.cond(A, 2)

re_A2 = LA.norm(delta_A, 2)/ LA.norm(A, 2)
re_b2 = LA.norm(delta_b, 2) / LA.norm(b, 2)

re_x2_ub = (re_A2 + re_b2) * kappa_A2 / (1 - kappa_A2 * re_A2)


print(f'The upper bound of the relative error of x in the L2 norm is {re_x2_ub.round(3)}.')

The upper bound of the relative error of x in the L2 norm is 0.094.


Using the L2 norm lowers the upper bound of relative error of x almost twice.

---
---

## Problem 3


In [11]:
A_hat = np.array([[16,-16],[-3,-1]])
b_hat = np.array([4,4])
x_hat = LA.inv(A_hat)@b_hat

print(f'Approximate solution: x = {x_hat[0]}, y = {x_hat[1]}')

Approximate solution: x = -0.9375, y = -1.1875


For the L2 norm:
$$\Delta b = \begin{pmatrix} \varepsilon_3 \\ \varepsilon_4 \end{pmatrix} \Rightarrow |\Delta b|_2 = \sqrt{\varepsilon_3^2 + \varepsilon_4^2} \le 0.05\sqrt{2}$$
$$\Downarrow$$
$$\delta b = \dfrac{|\Delta b|_2}{|b|_2} \le \dfrac{0.05\sqrt{2}}{4\sqrt{2}} = 0.0125$$

$$\Delta A = \begin{pmatrix} -4\varepsilon_1 & 4\varepsilon_2 \\ 0 & \varepsilon_1 \end{pmatrix} \Rightarrow ||\Delta A||_2 = \sigma_{max}(\Delta A^T\Delta A) = \sqrt{\frac{17\varepsilon_1^2 + \varepsilon_2^2 + \sqrt{(15\varepsilon_1^2 + \varepsilon_2^2)^2 + 4\varepsilon_1^2\varepsilon_2^2}}{2}} \le 0.207 \,(\text{lower bound is 0)}$$
$$\Downarrow$$
$$\delta A = \dfrac{||\Delta A||_2}{||A||_2} \le \dfrac{0.207}{8.283} = 0.025 \,(\text{lower bound is again 0)}$$

In [12]:
kappa_A_hat_L2 = LA.norm(A_hat, 2) * LA.norm(LA.inv(A_hat), 2)
# kappa_A_hat_2 = LA.cond(A_hat,2)

$$\delta x \le \dfrac{\kappa(A)}{1-\kappa(A)\delta A}(\delta A + \delta b)$$

In [13]:
re_x_L2 = kappa_A_hat_L2 * (0.0125 + 0.025) / (1 - kappa_A_hat_L2 * 0)
print(f'The upper bound of the relative error of x in the L2 norm is {re_x_L2.round(3)}.')

The upper bound of the relative error of x in the L2 norm is 0.301.


For the max norm:
$$\Delta b = \begin{pmatrix} \varepsilon_3 \\ \varepsilon_4 \end{pmatrix} \Rightarrow |\Delta b|_\infty \le 0.05$$
$$\Downarrow$$
$$\delta b = \dfrac{|\Delta b|_\infty}{|b|_\infty} \le \dfrac{0.05}{4} = 0.0125$$

$$\Delta A = \begin{pmatrix} -4\varepsilon_1 & 4\varepsilon_2 \\ 0 & \varepsilon_1 \end{pmatrix} \Rightarrow ||\Delta A||_\infty = max\{4\varepsilon_1, 4\varepsilon_2 + \varepsilon_1\} \le 5\cdot0.05 = 0.25$$
$$\Downarrow$$
$$\delta A = \dfrac{||\Delta A||_\infty}{||A||_\infty} \le \dfrac{0.25}{19} = 0.013$$

In [14]:
kappa_A_hat_Lmax = LA.norm(A_hat, np.inf) * LA.norm(LA.inv(A_hat), np.inf)
# kappa_A_hat_2 = LA.cond(A_hat,2)

re_x_Lmax = kappa_A_hat_Lmax * (0.0125 + 0.013) / (1 - kappa_A_hat_Lmax * 0)
print(f'The upper bound of the relative error of x in the max norm is {re_x_Lmax.round(3)}.')

The upper bound of the relative error of x in the max norm is 0.242.


The relative error of the solution is less when performing calculations in the max norm.

---
---

## Problem 4

$$\delta A^{-1} \le \frac{\kappa(A)}{1-\kappa(A)\delta A}\delta A, \quad \text{in the L1 norm}$$

In [15]:
A_hat = np.array([[9,-4],[-4,9]])
A_hat_inv = LA.inv(A_hat)
delta_A = 0.01 * np.ones((2,2))
re_A = LA.norm(delta_A, 1) / LA.norm(A_hat, 1)

kappa_A = LA.norm(A_hat, 1) * LA.norm(A_hat_inv, 1)
re_Ainv = kappa_A * re_A / (1 - kappa_A * re_A)

print(f'Approximation error wrt the L1 norm: {re_Ainv.round(3)}')

Approximation error wrt the L1 norm: 0.004


$$A^{-1} \approx \dfrac{1}{65} \cdot \begin{pmatrix} 9 & 4 \\ 4 & 9 \end{pmatrix} = \begin{pmatrix} 0.138 & 0.062 \\ 0.062 & 0.138 \end{pmatrix}$$

---
---

## Problem 5
Rewrite initial system in the following way:
$$Ax = B \Longleftrightarrow x_{k+1} = Px_k + b$$
Then if $x^*$ is the true solution, find an approximate one by iterating through the second system till $|x_k-x^*| \le \dfrac{|x_{k+1} - x_k|}{1-||P||} \le 0.01$. <br>
The starting point is a zero vector.

In [16]:
A = np.array([[22,1,6],[7,20,2],[1,2,21]])
B = np.array([9,1,2])
x_star = LA.inv(A)@B

In [17]:
P = np.array([[0,-1/22,-6/22],[-7/20,0,-2/20],[-1/21,-2/21,0]])
b = np.array([9/22,1/20,2/21])

P_norm = LA.norm(P, np.inf)
print(P_norm <= 1)

True


In [18]:
step = 0
x_old = np.zeros(3)
x_new = P@x_old + b
error = LA.norm(x_new - x_old, ord=np.inf) / (1 - P_norm)

while error > 0.01:
    x_old = x_new
    x_new = P@x_old + b
    error = LA.norm(x_new - x_old, ord=np.inf) / (1 - P_norm)
    step += 1

sol = x_old.round(3)
eps = LA.norm(x_old - x_star, np.inf).round(3)
print(f'Iterations: {step}', 
      f'Approximate solution: x = {sol[0]}, y = {sol[1]}, z = {sol[2]}', 
      f'Approximation error: {eps}', sep='\n')

Iterations: 4
Approximate solution: x = 0.39, y = -0.097, z = 0.085
Approximation error: 0.002


---
---

## Problem 6
Given a graph adjacency matrix A construct the probability matrix P and also the matrix Q of equal probabilities (ones over dimension), $\alpha$ is a dumping factor <br>
$$\tilde{P} = \alpha \cdot P + (1-\alpha) \cdot Q$$

Then iterate over a range of powers k in order to obtain some convergent solution:
$$X_{k+1} = \tilde{P} \cdot X_k$$

The most influential vertex will correspond to the index of the greatest value of the found PageRank vector X.

In [9]:
alpha = 0.85
A = np.array([[0,0,0,0,1],[1,0,1,0,1],[0,1,1,1,1],[0,0,0,1,1],[1,0,0,1,0]]).T
P = A / A.sum(axis=0)
Q = np.ones((5,5)) / 5
Ptilde = alpha * P + (1 - alpha) * Q
print(Ptilde)

[[0.03       0.31333333 0.03       0.03       0.455     ]
 [0.03       0.03       0.2425     0.03       0.03      ]
 [0.03       0.31333333 0.2425     0.03       0.03      ]
 [0.03       0.03       0.2425     0.455      0.455     ]
 [0.88       0.31333333 0.2425     0.455      0.03      ]]


In [10]:
X0 = np.array([[1/5],[1/5],[1/5],[1/5],[1/5]])
X = X0
for i in range(1000):
    X = Ptilde.dot(X)
    
print(f'PageRank vector: {np.ravel(X)}', 
      f'The most influential vertex: {np.argmax(X) + 1}', sep='\n')

PageRank vector: [0.19729979 0.04124893 0.05293612 0.34236787 0.36614729]
The most influential vertex: 5


---
---

## Problem 7
$$f(A) = e^{A+2\cdot I} = \sum \limits_{k=0}^{\infty} \frac{(A+2\cdot I)^k}{k!} = \left[\text{apply diagonalization:}\, A+2\cdot I = VDV^{-1} \right] = I + \frac{VDV^{-1}}{1!} + \frac{VDV^{-1}VDV^{-1}}{2!} + \frac{VDV^{-1}VDV^{-1}VDV^{-1}}{3!} + \dots = I + \frac{VDV^{-1}}{1!} + \frac{VD^2V^{-1}}{2!} + \frac{VD^3V^{-1}}{3!} + \dots = V \left(\sum \limits_{k=0}^{\infty} \frac{D^k}{k!}\right)V^{-1} = V\begin{pmatrix} \sum \limits_{k=0}^{\infty} \frac{\lambda_1^k}{k!} & 0 & 0 \\ 0 & \sum \limits_{k=0}^{\infty} \frac{\lambda_2^k}{k!} & 0 \\ 0 & 0 & \sum \limits_{k=0}^{\infty} \frac{\lambda_3^k}{k!} \end{pmatrix} V^{-1} = V \begin{pmatrix} e^{\lambda_1} & 0 & 0 \\ 0 & e^{\lambda_2} & 0 \\ 0 & 0 & e^{\lambda_3} \end{pmatrix} V^{-1}$$

In [21]:
A = np.array([[20,-5,6],[59,-15,21],[-5,1,1]])
A_plus_2 = A + 2*np.eye(3)

In [22]:
# make sure that the obtained matrix is diagonalizable
print(Matrix(A_plus_2).is_diagonalizable())

True


In [23]:
def exp_matrix(A):
    lam, V = LA.eig(A)
    exp_lam = np.exp(lam)
    exp_D = np.diag(exp_lam)
    
    return V@exp_D@LA.inv(V)

In [24]:
funcA = exp_matrix(A_plus_2)
matrix_str = "\\begin{pmatrix}\n"
matrix_str += "\\\\\n".join([" & ".join(map(str, row)) for row in np.round(funcA, 3)])
matrix_str += "\n\\end{pmatrix}"
display(Math(fr'f(A) = {matrix_str}'))

<IPython.core.display.Math object>

Let's check the result using the built-in `scipy` function `expm()`.

In [25]:
assert_almost_equal(funcA, expm(A_plus_2), decimal=3)